---
layout: post
title: Double Pendulum
---

In [1]:
from pathlib import Path
import os
import functools

from IPython.display import HTML, Image
import matplotlib.pyplot as plt
import numpy as np
from celluloid import Camera
import matplotlib.patches as patches
from scipy.integrate import odeint, solve_ivp
from simple_pid import PID

ROOT = Path("./assets/img/")

if not os.path.exists(ROOT):
    os.makedirs(ROOT)

A double pendulum.

In [2]:
L1 = 1.
L2 = 1.
m1 = 1.
m2 = 1.

XMIN = -L1-L2
XMAX = L1+L2
YMIN = -L1-L2
YMAX = 0.1

In [13]:
def show_fig(fig, tag):
    png_path = ROOT / f"{tag}.png"  
    fig.savefig(png_path)
    return Image(url=png_path)

In [16]:
def plot_pend(theta1, theta2, ax=None, png_path=None):

    if ax is None:
        ax = plt.subplot()

    x_m1 = L1*np.sin(theta1)
    y_m1 = -L1*np.cos(theta1)

    x_m2 = x_m1 + L2*np.sin(theta2)
    y_m2 = y_m1 - L2*np.cos(theta2)

    ax.set_xticks([])
    ax.set_yticks([])
    
    ax.set_xlim((XMIN, XMAX))
    ax.set_ylim((YMIN, YMAX))

    ax.hlines(0., xmin=XMIN, xmax=XMAX, color="black", linestyles="dashed")
    ax.vlines(0., ymin=YMIN, ymax=YMAX, color="black", linestyles="dashed")
    
    ax.plot([0, x_m1], [0, y_m1], color="black")
    ax.plot([x_m1, x_m2], [y_m1, y_m2], color="black")
    ax.scatter([x_m1, x_m2], [y_m1, y_m2], color="black")

    fig = ax.get_figure()
    plt.close()

    return fig

In [17]:
show_fig(plot_pend(theta1=1., theta2=2.), tag="double_pend")

Let $L_1$ and $L_2$ be the lengths of the upper and lower arms, respectively, $m_1$ and $m_2$ the upper and lower masses, $\theta_1$ and $\theta_2$ the angles between the upper and lower arms and the vertical, $g = -9.81$ the gravity acceleration, $x_1$, $y_1$, $x_2$, $y_2$ the $x,y$ coordinates of the upper and lower masses.

$$
\begin{align*}
T \cos(\theta_1) + m_1 g = m_1 \ddot{y}_1\\\\
-T \sin(\theta_1) = m_1 \ddot{x}_1\\\\
-T \cos(\theta_2) + m_2 g = m_2 \ddot{y}_2\\\\
T \sin(\theta_2) = m_2 \ddot{x}_2
\end{align*}
$$

Eliminating $T$: 

$$
-m_1 \ddot{x}_1 \cos(\theta_1) + m_1 g \sin(\theta_1) = m_1 \ddot{y}_1 \sin(\theta_1)
$$

$$
-m_2 \ddot{x}_2 \cos(\theta_2) + m_2 g \sin(\theta_2) = m_2 \ddot{y}_2 \sin(\theta_2)
$$

$$
-m_1 \ddot{x}_1 \sin(\theta_2) = m_2 \ddot{x}_2 \sin(\theta_1)
$$

Using:

$$
\begin{align*}
x_2 = x_1 + L_2 \sin(\theta_2)\\\\
y_2 = y_1 - L_2 \cos(\theta_2)
\end{align*}
$$

We have:

$$
\ddot{x}_2 = \ddot{x}_1 + L_2 (\cos(\theta_2) \ddot{\theta}_2 - \sin(\theta_2) \dot{\theta}_2^2)
$$

$$
\ddot{y}_2 = \ddot{y}_1 + L_2 (\sin(\theta_2)\ddot{\theta}_2 + \cos(\theta_2)\dot{\theta}_2^2)
$$

Substituting:

$$
m_2 \ddot{x}_1 \cos(\theta_2) + m_2 \ddot{y}_1 \sin(\theta_2) + m_2 L_2 \ddot{\theta}_2 = m_2 g \sin(\theta_2)
$$

$$
(m_1 \sin(\theta_2) + m_2 \sin(\theta_1))\ddot{x}_1 + m_2 L_2 \sin(\theta_1) \cos(\theta_2) \ddot{\theta}_2 = m_2 L_2 \sin(\theta_1) \sin(\theta_2) \dot{\theta}_2^2
$$

$$
-m_1 \ddot{x}_1 \cos(\theta_1) + m_1 g \sin(\theta_1) = m_1 \ddot{y}_1 \sin(\theta_1)
$$